In [2]:
import numpy as np


class Perceptron(object):
    """Perceptron classifier.(パーセプトロン分類器)

    Parameters
    ------------
    eta : float
      Learning rate (between 0.0 and 1.0)
    n_iter : int
      Passes over the training dataset.(トレーニング回数)
    random_state : int
      Random number generator seed for random weight
      initialization.

    Attributes
    -----------
    w_ : 1d-array
      Weights after fitting.
    errors_ : list
      Number of misclassifications (updates) in each epoch.

    """
    #初期化
    def __init__(self, eta=0.01, n_iter=50, random_state=1):
        self.eta = eta
        self.n_iter = n_iter
        self.random_state = random_state

    def fit(self, X, y):
        """Fit training data.

        Parameters
        ----------
        X : {array-like}, shape = [n_samples, n_features]
          Training vectors, where n_samples is the number of samples(サンプル個数) and
          n_features is the number of features.(特徴量の個数)
        y : array-like, shape = [n_samples]
          Target values.

        Returns
        -------
        self : object

        """
        #乱数生成
        rgen = np.random.RandomState(self.random_state)
        #重みベクトルの初期値設定
        self.w_ = rgen.normal(loc=0.0, scale=0.01, size=1 + X.shape[1])
        #エラー記録用配列
        self.errors_ = []

        #指定した繰り返し回数分、学習を繰り返す
        for _ in range(self.n_iter):
            #エラー初期化
            errors = 0
            #各サンプルで重み更新
            for xi, target in zip(X, y):
                #更新量計算
                update = self.eta * (target - self.predict(xi))
                #更新
                self.w_[1:] += update * xi
                self.w_[0] += update
                #エラーカウント
                errors += int(update != 0.0)
            #エラー記録
            self.errors_.append(errors)
        return self

    #総入力を計算
    def net_input(self, X):
        """Calculate net input"""
        return np.dot(X, self.w_[1:]) + self.w_[0]

    #学習した分類器を元に、予測
    def predict(self, X):
        """Return class label after unit step"""
        return np.where(self.net_input(X) >= 0.0, 1, -1)
    

In [3]:
class LogisticRegressionGD(object):
    """Logistic Regression Classifier using gradient descent.(勾配降下法に基づくロジスティック回帰分類器)
    

    Parameters
    ------------
    eta : float
      Learning rate (between 0.0 and 1.0)
    n_iter : int
      Passes over the training dataset.
    random_state : int
      Random number generator seed for random weight
      initialization.


    Attributes
    -----------
    w_ : 1d-array
      Weights after fitting.
    cost_ : list
      Logistic cost function value in each epoch.

    """
    def __init__(self, eta=0.05, n_iter=100, random_state=1):
        self.eta = eta
        self.n_iter = n_iter
        self.random_state = random_state

    def fit(self, X, y):
        """ Fit training data.

        Parameters
        ----------
        X : {array-like}, shape = [n_samples, n_features]
          Training vectors, where n_samples is the number of samples and
          n_features is the number of features.
        y : array-like, shape = [n_samples]
          Target values.

        Returns
        -------
        self : object

        """
        rgen = np.random.RandomState(self.random_state)
        self.w_ = rgen.normal(loc=0.0, scale=0.01, size=1 + X.shape[1])
        self.cost_ = []

        for i in range(self.n_iter):
            net_input = self.net_input(X)
            output = self.activation(net_input)
            errors = (y - output)
            self.w_[1:] += self.eta * X.T.dot(errors)
            self.w_[0] += self.eta * errors.sum()
            
            # note that we compute the logistic `cost` now
            # instead of the sum of squared errors cost
            cost = -y.dot(np.log(output)) - ((1 - y).dot(np.log(1 - output)))
            self.cost_.append(cost)
        return self
    
    def net_input(self, X):
        """Calculate net input"""
        return np.dot(X, self.w_[1:]) + self.w_[0]

    def activation(self, z):
        """Compute logistic sigmoid activation"""
        return 1. / (1. + np.exp(-np.clip(z, -250, 250)))

    def predict(self, X):
        """Return class label after unit step"""
        return np.where(self.net_input(X) >= 0.0, 1, 0)
        # equivalent to:
        # return np.where(self.activation(self.net_input(X)) >= 0.5, 1, 0)
        